In [1]:
import numpy as np
import tqdm
import pandas as pd

In [2]:
supplychain = pd.read_csv("../assets/global_supplychain.csv")
supplychain["accounting_as_of_date"] = pd.to_datetime(supplychain.accounting_as_of_date)

# supplychain["accounting_as_of_date"] = supplychain["accounting_as_of_date"].dt.strftime('%m/%Y')
# supplychain.drop_duplicates(subset=["accounting_as_of_date", "supplier_ticker", "customer_ticker"], keep='first', inplace=True)

In [3]:
supplychain.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3074589 entries, 0 to 3074588
Data columns (total 9 columns):
Unnamed: 0                       int64
public_domain_date               object
accounting_as_of_date            datetime64[ns]
supplier_exchange                object
supplier_ticker                  object
customer_exchange                object
customer_ticker                  object
revenue_dependency               float64
revenue_dependency_annotation    object
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 140.7+ MB


In [4]:
stock_market = pd.read_pickle("../assets/filtered_us_eod.pkl")
stock_market = stock_market.drop(["Date.1"], axis=1)
stock_market["Date"] = pd.to_datetime(stock_market.Date)
stock_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4467840 entries, 0 to 13171391
Data columns (total 6 columns):
Date         datetime64[ns]
ticker       object
adj_close    float64
mom          float64
MACD         float64
vol          float64
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 221.6+ MB


In [5]:
# stock_market["Date"] = pd.to_datetime(stock_market.Date)
# stock_market["Date"] = stock_market["Date"].dt.strftime('%m/%Y')
# stock_market.drop_duplicates(subset=["Date", "ticker"], keep='first', inplace=True)

In [6]:
stock_market

,Date,ticker,adj_close,mom,MACD,vol
0,2010-01-01,A,20.822542,5.517390,3.395021,0.000097
1,2010-01-04,A,20.976683,5.344631,3.421450,0.000094
2,2010-01-05,A,20.748822,4.587213,3.431355,0.000100
3,2010-01-06,A,20.675102,4.223339,3.434483,0.000098
4,2010-01-07,A,20.648294,4.011330,3.433929,0.000098
...,...,...,...,...,...,...
13171387,2019-10-14,ZIXI,6.750000,-3.205437,-0.218251,0.000565
13171388,2019-10-15,ZIXI,6.865000,-2.722324,-0.243142,0.000575
13171389,2019-10-16,ZIXI,6.650000,-3.055404,-0.278275,0.000600
13171390,2019-10-17,ZIXI,6.650000,-2.862559,-0.310736,0.000588


In [7]:
# check overlapping

companies = list(set(stock_market["ticker"].values.tolist())) # get rid of date
customers = supplychain["customer_ticker"].values.tolist()
suppliers = supplychain["supplier_ticker"].values.tolist()


customer_suppliers = customers + suppliers
customer_suppliers = set(customer_suppliers)
print(len(customer_suppliers))
print(len(set(customers)))
print(len(set(suppliers)))
# customers = np.sort(customers)
# suppliers = np.sort(suppliers)

23854
19313
7974


In [8]:
# calculate overlaps

overlap = [x for x in tqdm.tqdm(companies) if x in customer_suppliers]
print(len(overlap))
set_customers = set(customers)
set_suppliers = set(suppliers)
overlap_customers = [x for x in tqdm.tqdm(companies) if x in set_customers]
print(len(overlap_customers))
overlap_suppliers = [x for x in tqdm.tqdm(companies) if x in set_suppliers]
print(len(overlap_suppliers))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1790/1790 [00:00<00:00, 897525.90it/s]


1790


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1790/1790 [00:00<00:00, 1800864.51it/s]


933


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1790/1790 [00:00<00:00, 897096.92it/s]


1403


In [9]:
# remove non overlapped on stock market
non_overlapping_companies = [x for x in companies if x not in overlap]
# print(len(non_overlapping_companies))

print("Before(Market):", stock_market.shape)
stock_market_updated = stock_market.drop(non_overlapping_companies, axis=1)
print(stock_market_updated.shape)
print("After(Market):", stock_market_updated.shape)

# remove non overlapped on supply chain

print("Before(Supply Chain):", supplychain.shape)
supplychain_updated = supplychain[supplychain.supplier_ticker.isin(overlap_suppliers)]
supplychain_updated = supplychain_updated[supplychain_updated.customer_ticker.isin(overlap_customers)]
print("After(Supply Chain):", supplychain_updated.shape)

Before(Market): (4467840, 6)
(4467840, 6)
After(Market): (4467840, 6)
Before(Supply Chain): (3074589, 9)
After(Supply Chain): (336934, 9)


In [10]:
with open('../assets/overlapping_companies.txt', 'w') as f:
    for item in overlap:
        f.write("%s\n" % item)

In [11]:
stock_market_updated.set_index("Date")
stock_market_updated

MemoryError: Unable to allocate array with shape (4, 4467840) and data type float64

In [ ]:
supplychain_updated.set_index("accounting_as_of_date")
supplychain_updated = supplychain_updated[supplychain_updated["accounting_as_of_date"] >= stock_market_updated["Date"].min()]
supplychain_updated = supplychain_updated.drop("Unnamed: 0", axis=1)
supplychain_updated.drop_duplicates(subset=["accounting_as_of_date", "supplier_ticker", "customer_ticker"], keep="first", inplace=True)
supplychain_updated

In [ ]:
# account_dates = supplychain_updated["accounting_as_of_date"].values
# for company in tqdm.tqdm(overlap_suppliers):
#     for date in set(stock_market_updated["Date"].unique()):
# #             print(supplychain_updated.loc[(supplychain_updated['supplier_ticker'] == company) &
# #                                    (supplychain_updated['accounting_as_of_date'] == date), "revenue_dependency"])
#         stock_market_updated.loc[(stock_market_updated['ticker'] == company) &
#                                (stock_market_updated['Date'] == date), "revenue_dependency"] = \
#         sum(supplychain_updated.loc[(supplychain_updated['supplier_ticker'] == company) &
#                                (supplychain_updated['accounting_as_of_date'] == date), "revenue_dependency"])
#     break

supplychain_updated

grouped_supplychain = supplychain_updated.groupby(['accounting_as_of_date', 'supplier_ticker'])

revenue_sum = grouped_supplychain.agg({'revenue_dependency': 'sum'})

joined_market_info = stock_market_updated.merge(revenue_sum, left_on=["Date", "ticker"], right_on=["accounting_as_of_date", "supplier_ticker"], how="left").fillna(method="ffill")

joined_market_info
            
# joined_market_info = stock_market_updated.merge(supplychain_updated, left_on=["Date", "ticker"], right_on=["accounting_as_of_date", "supplier_ticker"])
    
# joined_market_info
    
    

In [ ]:
stock_market_updated.loc[(stock_market_updated['ticker'] == companies[0]) &
                               (stock_market_updated['Date'] == set(stock_market_updated["Date"].unique())[0]), "revenue_dependency"]

In [ ]:
[x for x in stock_market_updated["revenue_dependency"].values if x > -1]